# Neste notebook, faremos a seleção e filtragem dos dados para treino dos modelos e subsequente teste.

# Os dados processados serão salvos em /dados/dados processados/




In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Dissertação Caio

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Dissertação Caio


In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt
from parametros_OM import Xi_y, m_o, Ca_pot, q_hat, C_dv, C_0l, St, c

# Parte 1: seleção dos dados

In [ ]:
prefix = './dados/dados experimentais/Data2025mstar26number'
prefixU = './dados/dados experimentais/Ur2025mstar26number'

np.random.seed(42)

all_idxs = list(5*np.random.permutation(list(range(38))) + 95)
all_idxs.remove(190)
train_idxs = all_idxs[:25]
test_idxs = all_idxs[25:]

n_train = len(train_idxs)
n_test = len(test_idxs)

T = np.zeros((len(train_idxs), 18000))
Y = np.zeros((len(train_idxs), 18000))
U = np.zeros(len(train_idxs))

for i, idx in enumerate(train_idxs):
    if idx == 190:
        pass
    else:
      dados = np.loadtxt(prefix + str(idx), delimiter=',')
      dadosU = np.loadtxt(prefixU + str(idx))
      T[i, :] = dados[:, 0]  # 1ª coluna (índice 0)
      Y[i, :] = dados[:, 1] # 2ª coluna (índice 1)
      U[i] = np.mean(dadosU)

# Parte 2: filtragem

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter(order, cutoff, btype='low', fs=fs, analog=False)
    y = filtfilt(b, a, data)
    return y

# parametros adotados para o filtro
cutoff = .4
order = 4

In [9]:
partes = 10
Y_pd = np.zeros((n_train, 18000))
Y_sd = np.zeros((n_train, 18000))
Y_filtered = np.zeros((n_train, 18000))
Y_pd_filtered = np.zeros((n_train, 18000))
Y_sd_filtered = np.zeros((n_train, 18000))
Q_filtered = np.zeros((n_train, 18000))
Q_pd_filtered = np.zeros((n_train, 18000))
Q_sd_filtered = np.zeros((n_train, 18000))

for i in range(n_train):
      t = T[i,:]
      y = Y[i,:]
      Ur = U[i]
      S = St*Ur
      fs = 1 / (t[1] - t[0])
      y_filtered = butter_lowpass_filter(y, cutoff, fs, order)
      y_pd = np.gradient(y, t)
      y_filtered_pd_pre = np.gradient(y_filtered, t)
      y_filtered_pd = butter_lowpass_filter(y_filtered_pd_pre, cutoff, fs, order)
      y_sd = np.gradient(y_pd, t)
      y_filtered_sd_pre = np.gradient(y_filtered_pd, t)
      y_filtered_sd = butter_lowpass_filter(y_filtered_sd_pre, cutoff, fs, order)
      a = y_sd + 2*Xi_y*y_pd + y
      b = (1/(2*np.pi**3))*(Ur**2/(m_o+Ca_pot))
      d = 2*np.pi*C_dv*y_pd/Ur
      e = np.sqrt(1 + (2*np.pi*y_pd/Ur)**2)
      q = (a/b + e*d)/(e*c)
      q_filtered = butter_lowpass_filter(q, cutoff, fs, order)
      q_pd = np.gradient(q, t)
      q_filtered_pd_pre = np.gradient(q_filtered, t)
      q_filtered_pd = butter_lowpass_filter(q_filtered_pd_pre, cutoff, fs, order)
      q_sd = np.gradient(q_pd, t)
      q_filtered_sd_pre = np.gradient(q_filtered_pd, t)
      q_filtered_sd = butter_lowpass_filter(q_filtered_sd_pre, cutoff, fs, order)
      Y_pd[i,:] = y_pd
      Y_sd[i,:] = y_sd
      Y_filtered[i,:] = y_filtered
      Y_pd_filtered[i,:] = y_filtered_pd
      Y_sd_filtered[i,:] = y_filtered_sd
      Q_filtered[i,:] = q_filtered
      Q_pd_filtered[i,:] = q_filtered_pd
      Q_sd_filtered[i,:] = q_filtered_sd

Y_filtered_parts = np.split(Y_filtered, partes, axis=1)
Y_pd_filtered_parts = np.split(Y_pd_filtered, partes, axis=1)
Y_sd_filtered_parts = np.split(Y_sd_filtered, partes, axis=1)
Q_filtered_parts = np.split(Q_filtered, partes, axis=1)
Q_pd_filtered_parts = np.split(Q_pd_filtered, partes, axis=1)
Q_sd_filtered_parts = np.split(Q_sd_filtered, partes, axis=1)

In [10]:
np.save('./dados/dados processados/treino/U', U)
np.save('./dados/dados processados/treino/T', T)
np.save('./dados/dados processados/treino/Y', Y)
np.save('./dados/dados processados/treino/Y_pd', Y_pd)
np.save('./dados/dados processados/treino/Y_sd', Y_sd)
np.save('./dados/dados processados/treino/Y_filtered', Y_filtered)
np.save('./dados/dados processados/treino/Y_pd_filtered', Y_pd_filtered)
np.save('./dados/dados processados/treino/Y_sd_filtered', Y_sd_filtered)
np.save('./dados/dados processados/treino/Y_filtered_parts', Y_filtered_parts)
np.save('./dados/dados processados/treino/Y_pd_filtered_parts', Y_pd_filtered_parts)
np.save('./dados/dados processados/treino/Y_sd_filtered_parts', Y_sd_filtered_parts)
np.save('./dados/dados processados/treino/Q_filtered', Q_filtered)
np.save('./dados/dados processados/treino/Q_pd_filtered', Q_pd_filtered)
np.save('./dados/dados processados/treino/Q_sd_filtered', Q_sd_filtered)
np.save('./dados/dados processados/treino/Q_filtered_parts', Q_filtered_parts)
np.save('./dados/dados processados/treino/Q_pd_filtered_parts', Q_pd_filtered_parts)
np.save('./dados/dados processados/treino/Q_sd_filtered_parts', Q_sd_filtered_parts)